In [1]:
#Might need to adjust IP in youtube API
import requests as rq
from bs4 import BeautifulSoup
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Construct URL
all_url = []
temp = 'http://www.boxofficemojo.com/alltime/world/?pagenum'
#temp
for i in range(8):
    all_url.append(temp + '=' + str(i+1) + '&p=.htm')
    #print(temp + '=' + str(i+1) + '&p=.htm' )
all_url

['http://www.boxofficemojo.com/alltime/world/?pagenum=1&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=2&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=3&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=4&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=5&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=6&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=7&p=.htm',
 'http://www.boxofficemojo.com/alltime/world/?pagenum=8&p=.htm']

In [3]:
#This chunk if for scraping name and revenue
name = []
rev = []

for k in all_url:
    r = rq.get(k)
    r
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    type(soup)
    a = soup.findAll('b')
    a = a[3:len(a)-3]
    for i in range(len(a)):
        if i%2 == 0:
            name.append(a[i].get_text())
        else:
            rev.append(float(a[i].get_text()[1:].replace(",","")))
    
name = np.array(name).reshape(-1,1)
rev = np.array(rev).reshape(-1,1)

In [4]:
df = pd.DataFrame({'name' :name.ravel()  ,
                              'rev': rev.ravel().astype('float')})
df.head()

,name,rev
0,Avatar,2788.0
1,Titanic,2187.5
2,Star Wars: The Force Awakens,2068.2
3,Jurassic World,1671.7
4,Marvel's The Avengers,1518.8


In [24]:
#Getting more detail
#link_text = [a['href'] for a in soup.find_all('a', href=True) if a.text.strip()]
#len(link_text)
#for j in range(38 , len(link_text) - 27):
#print(link_text[j])
distributor = []
genre = []
rating  = []
budget = []
for url in all_url:
    r = rq.get(url)
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    type(soup)
    a = soup.findAll('a' , href = True)
    link_text = [a['href'] for a in soup.find_all('a', href=True) if a.text.strip()]
    for m in range(38,len(link_text)-27):
        #print(link_text[m])
        new_url =  "http://www.boxofficemojo.com" + link_text[m]
        r = rq.get(new_url)
        html = r.text
        soup = BeautifulSoup(html, "html5lib")
        a = soup.find("table" , {"style" : "padding-top: 5px;"}  )
        children = a.findAll("td" , {"valign" : "top"})
        
        distributor.append(children[2].get_text())
        genre.append(children[4].get_text())
        rating.append(children[6].get_text())
        budget.append(children[7].get_text())

In [25]:
genre_new , Distributor_new , rating_new , budget_new = [] , [] , [] , []
for i in range(len(genre)):
    genre_new.append(genre[i][genre[i].find(":")+1:])
    Distributor_new.append(distributor[i][distributor[i].find(":")+1:])
    rating_new.append(rating[i][rating[i].find(":")+1:])
    budget_new.append(budget[i][budget[i].find("$")+1:].split()[0])

genre_new = np.array(genre_new)
Distributor_new = np.array(Distributor_new)
rating_new = np.array(rating_new)
budget_new = np.array(budget_new)
budget_new[np.where(budget_new== 'Production')] = "NaN"


In [26]:
#dat = pd.DataFrame(genre_new ,  Distributor_new , rating_new  )
df['genre'] =  genre_new.ravel() 
df['Distributor'] =  Distributor_new.ravel()
df['Rating'] =  rating_new.ravel()
df['Budget'] = budget_new.ravel()
df['Budget'].replace('NaN' , np.NaN)
df['Budget'] = pd.to_numeric(df['Budget'] , errors='coerce')
#budget_new.replace('NaN', np.NaN)
df.head()

,name,rev,view,like,dislike,comment,genre,Distributor,Rating,Budget
0,Avatar,2788.0,19929824,22396,2431,10455,Sci-Fi Adventure,Fox,PG-13,237.0
1,Titanic,2187.5,16855757,39943,1823,10586,Romance,Paramount,PG-13,200.0
2,Star Wars: The Force Awakens,2068.2,185150509,1097369,44323,209184,Sci-Fi Fantasy,Buena Vista,PG-13,245.0
3,Jurassic World,1671.7,133246187,927462,47529,126958,Action / Adventure,Universal,PG-13,150.0
4,Marvel's The Avengers,1518.8,143928901,2903270,62566,397740,Action / Adventure,Buena Vista,PG-13,220.0


In [27]:
#trying to get year here
year = []
for k in all_url:
    r = rq.get(k)
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    a = soup.findAll('td' , {"align" : "center" } , {"size" : "2"})
    for i in range(9 , len(a),  2):
        temp = (a[i].get_text())[0:4]
        year.append(temp)

In [28]:
df['Year'] = year
df.head()


,name,rev,view,like,dislike,comment,genre,Distributor,Rating,Budget,Year
0,Avatar,2788.0,19929824,22396,2431,10455,Sci-Fi Adventure,Fox,PG-13,237.0,2009
1,Titanic,2187.5,16855757,39943,1823,10586,Romance,Paramount,PG-13,200.0,1997
2,Star Wars: The Force Awakens,2068.2,185150509,1097369,44323,209184,Sci-Fi Fantasy,Buena Vista,PG-13,245.0,2015
3,Jurassic World,1671.7,133246187,927462,47529,126958,Action / Adventure,Universal,PG-13,150.0,2015
4,Marvel's The Avengers,1518.8,143928901,2903270,62566,397740,Action / Adventure,Buena Vista,PG-13,220.0,2012


In [5]:
#Get youtube data 
# Scraping Youtube View 

#sys.path.remove("C:\\Users\\USA\\youtube_tutorial")
from apiclient.discovery import build #pip install google-api-python-client
from apiclient.errors import HttpError #pip install google-api-python-client
from oauth2client.tools import argparser #pip install oauth2client
import sys
sys.path.append("C:\\Users\\USA\\youtube_tutorial")
from youtube_videos import youtube_search
import json
sys.path

['',
 'C:\\Users\\USA\\Anaconda3\\python36.zip',
 'C:\\Users\\USA\\Anaconda3\\DLLs',
 'C:\\Users\\USA\\Anaconda3\\lib',
 'C:\\Users\\USA\\Anaconda3',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\Sphinx-1.5.6-py3.6.egg',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\setuptools-36.6.0-py3.6.egg',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\USA\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\USA\\.ipython',
 'C:\\Users\\USA\\youtube_tutorial']

In [112]:
#sys.path.remove("C:\\Users\\USA\\youtube_tutorial")
#sys.path.remove("C:\\Users\\USA\\pafy\\pafy")


ValueError: list.remove(x): x not in list

In [14]:
from youtube_videos import geo_query

viewCount = likeCount = dislikeCount = commentCount = 0 
viewCount_store  = []
likeCount_store = [] 
dislikeCount_store = [] 
commentCount_store = []

for idx , movie in enumerate(name):
    print(idx , movie)
    test = youtube_search(str(movie)[2:-2] + " Trailers")
    #Extract only top three result
    for video in test[1][0:3]:
        #print (video['id']['videoId'])
        k = geo_query(video['id']['videoId'])
       # print(k['items'][0]['statistics'])
        #print(viewCount)
        try:
            viewCount +=   int(k['items'][0]['statistics']['viewCount'])
            likeCount +=  int(k['items'][0]['statistics']['likeCount'])
            dislikeCount += int(k['items'][0]['statistics']['dislikeCount'])
            commentCount += int(k['items'][0]['statistics']['commentCount'])
        except Exception:
            likeCount +=  0
            dislikeCount += 0
            commentCount += 0
        
    #print(viewCount)
    viewCount_store.append(viewCount)
    #print(viewCount_store)
    likeCount_store.append(likeCount)
    dislikeCount_store.append(dislikeCount)
    commentCount_store.append(commentCount)
    viewCount = 0 
    likeCount = 0 
    dislikeCount = 0 
    commentCount = 0
        #dislikeCount += dislikeCount + int(k['items'][0]['statistics']['dislikeCount'])
        
        #print(commentCount)
        #print(likeCount)/        #print(k['items'][0]['statistics'])

#test


0 ['Avatar']
1 ['Titanic']
2 ['Star Wars: The Force Awakens']
3 ['Jurassic World']
4 ["Marvel's The Avengers"]
5 ['Furious 7']
6 ['Avengers: Age of Ultron']
7 ['Harry Potter and the Deathly Hallows Part 2']
8 ['Frozen']
9 ['Beauty and the Beast (2017)']
10 ['The Fate of the Furious']
11 ['Iron Man 3']
12 ['Minions']
13 ['Captain America: Civil War']
14 ['Transformers: Dark of the Moon']
15 ['The Lord of the Rings: The Return of the King']
16 ['Skyfall']
17 ['Transformers: Age of Extinction']
18 ['The Dark Knight Rises']
19 ['Toy Story 3']
20 ["Pirates of the Caribbean: Dead Man's Chest"]
21 ['Star Wars: The Last Jedi']
22 ['Rogue One: A Star Wars Story']
23 ['Pirates of the Caribbean: On Stranger Tides']
24 ['Despicable Me 3']
25 ['Jurassic Park']
26 ['Finding Dory']
27 ['Star Wars: Episode I - The Phantom Menace']
28 ['Alice in Wonderland (2010)']
29 ['Zootopia']
30 ['The Hobbit: An Unexpected Journey']
31 ['The Dark Knight']
32 ["Harry Potter and the Sorcerer's Stone"]
33 ['Despicabl

296 ['The Fugitive']
297 ['Jurassic Park III']
298 ['My Big Fat Greek Wedding']
299 ['Hitch']
300 ['Shark Tale']
301 ['Die Hard: With A Vengeance']
302 ['Notting Hill']
303 ["A Bug's Life"]
304 ['Night at the Museum: Secret of the Tomb']
305 ['Fast and Furious']
306 ["Ocean's Twelve"]
307 ['Noah']
308 ['Planet of the Apes (2001)']
309 ['The Hangover Part III']
310 ['The World Is Not Enough']
311 ['Alvin and the Chipmunks']
312 ['The Karate Kid']
313 ['Home Alone 2: Lost in New York']
314 ['Hotel Transylvania']
315 ['Minority Report']
316 ['Top Gun']
317 ['The Legend of Tarzan']
318 ['American Beauty']
319 ['Your Name.']
320 ['Apollo 13']
321 ['Rain Man']
322 ['The Great Gatsby (2013)']
323 ['X-Men: First Class']
324 ['Basic Instinct']
325 ['GoldenEye']
326 ['Catch Me If You Can']
327 ['Now You See Me']
328 ['Hannibal']
329 ['The Mask']
330 ['Speed']
331 ['Dinosaur']
332 ['The Angry Birds Movie']
333 ['Deep Impact']
334 ["Dr. Seuss' The Lorax"]
335 ['The Maze Runner']
336 ['The Smurfs 2

617 ['Yes Man']
618 ['Non-Stop']
619 ['Honey, I Shrunk the Kids']
620 ['Fahrenheit 9/11']
621 ['Jumper']
622 ['Wild Wild West']
623 ['Presumed Innocent']
624 ['Scary Movie 3']
625 ['Elf']
626 ['Inferno']
627 ['Green Lantern']
628 ['The Terminal']
629 ['What Happens in Vegas']
630 ["It's Complicated"]
631 ['Knocked Up']
632 ['Captain Phillips']
633 ['The Prince of Egypt']
634 ['Jack Reacher']
635 ['Collateral']
636 ['Dracula Untold']
637 ['The Emoji Movie']
638 ['Central Intelligence']
639 ['Million Dollar Baby']
640 ['Ted 2']
641 ['The Help']
642 ['Twins']
643 ["Valentine's Day"]
644 ['Bad Teacher']
645 ['Clear and Present Danger']
646 ['101 Dalmatians']
647 ["Bram Stoker's Dracula"]
648 ['Vertical Limit']
649 ['Ghostbusters II']
650 ['The Patriot']
651 ["The Sorcerer's Apprentice"]
652 ['Just Go With It']
653 ['The Expendables 3']
654 ['Public Enemies']
655 ['The Bourne Identity']
656 ['Disclosure']
657 ['Dirty Dancing']
658 ['Pulp Fiction']
659 ['The Aviator']
660 ['Crouching Tiger, 

In [33]:
df['view'] , df['like'] , df['dislike'] , df['comment']= viewCount_store , likeCount_store , dislikeCount_store , commentCount_store
df['Year'] = pd.to_numeric(df['Year'])
df.dtypes

name            object
rev            float64
view             int64
like             int64
dislike          int64
comment          int64
genre           object
Distributor     object
Rating          object
Budget         float64
Year             int64
dtype: object

In [35]:
df.head()
df.to_csv('all_data_movies.csv')

In [37]:
from imdb import IMDb

In [124]:
import time
rating_avg = []
m_temp = []
ia = IMDb()
for mov in df['name']:    
    try:
            movies = ia.search_movie(mov)
    except Exception:
            m_temp.append(mov)
            rating_avg.append(0)
    
    try:
            movies = ia.search_movie(mov) 
            item = movies[0]
            ia.update(item)
            m_temp.append(mov)
            rating_avg.append(item['rating'])
    except Exception:
            pass
    print("Movie at " , len(m_temp) , " Rating at ", len(rating_avg))
    #print(item['rating'])
    
    #time.sleep(np.random.random(1))

Movie at  1  Rating at  1
Movie at  2  Rating at  2
Movie at  3  Rating at  3
Movie at  4  Rating at  4
Movie at  5  Rating at  5
Movie at  6  Rating at  6
Movie at  7  Rating at  7
Movie at  8  Rating at  8
Movie at  9  Rating at  9
Movie at  10  Rating at  10
Movie at  11  Rating at  11
Movie at  12  Rating at  12
Movie at  13  Rating at  13
Movie at  14  Rating at  14
Movie at  15  Rating at  15
Movie at  16  Rating at  16
Movie at  17  Rating at  17
Movie at  18  Rating at  18
Movie at  19  Rating at  19
Movie at  20  Rating at  20
Movie at  21  Rating at  21
Movie at  22  Rating at  22
Movie at  23  Rating at  23
Movie at  24  Rating at  24
Movie at  25  Rating at  25
Movie at  26  Rating at  26
Movie at  27  Rating at  27
Movie at  28  Rating at  28
Movie at  29  Rating at  29
Movie at  30  Rating at  30
Movie at  31  Rating at  31
Movie at  32  Rating at  32
Movie at  33  Rating at  33
Movie at  34  Rating at  34
Movie at  35  Rating at  35
Movie at  36  Rating at  36
Movie at  

Movie at  281  Rating at  280
Movie at  282  Rating at  281
Movie at  283  Rating at  282
Movie at  284  Rating at  283
Movie at  285  Rating at  284
Movie at  286  Rating at  285
Movie at  287  Rating at  286
Movie at  288  Rating at  287
Movie at  289  Rating at  288
Movie at  290  Rating at  289
Movie at  291  Rating at  290
Movie at  292  Rating at  291
Movie at  293  Rating at  292
Movie at  294  Rating at  293
Movie at  295  Rating at  294
Movie at  296  Rating at  295
Movie at  297  Rating at  296
Movie at  298  Rating at  297
Movie at  299  Rating at  298
Movie at  300  Rating at  299
Movie at  301  Rating at  300
Movie at  302  Rating at  301
Movie at  303  Rating at  302
Movie at  304  Rating at  303
Movie at  305  Rating at  304
Movie at  306  Rating at  305
Movie at  307  Rating at  306
Movie at  308  Rating at  307
Movie at  309  Rating at  308
Movie at  310  Rating at  309
Movie at  311  Rating at  310
Movie at  312  Rating at  311
Movie at  313  Rating at  312
Movie at  

Movie at  555  Rating at  553
Movie at  556  Rating at  554
Movie at  557  Rating at  555
Movie at  558  Rating at  556
Movie at  559  Rating at  557
Movie at  560  Rating at  558
Movie at  561  Rating at  559
Movie at  562  Rating at  560
Movie at  563  Rating at  561
Movie at  564  Rating at  562
Movie at  565  Rating at  563
Movie at  566  Rating at  564
Movie at  567  Rating at  565
Movie at  568  Rating at  566
Movie at  569  Rating at  567
Movie at  570  Rating at  568
Movie at  571  Rating at  569
Movie at  572  Rating at  570
Movie at  573  Rating at  571
Movie at  574  Rating at  572
Movie at  575  Rating at  573
Movie at  576  Rating at  574
Movie at  577  Rating at  575
Movie at  578  Rating at  576
Movie at  579  Rating at  577
Movie at  580  Rating at  578
Movie at  581  Rating at  579
Movie at  582  Rating at  580
Movie at  583  Rating at  581
Movie at  584  Rating at  582
Movie at  585  Rating at  583
Movie at  586  Rating at  584
Movie at  587  Rating at  585
Movie at  

In [76]:

item = movies[0]

#item
#ia.update(item)
#ia.update(item , 'goofs')
print(item.keys())
print(item['votes'])
print(item['plot outline'])
print(item['votes'])
print(item['title'])
#print(item['median'])

#for item in movies:
#print(item['long imdb canonical title'], item.movieID)

#for item in movies:
# print(item['rating'])


['title', 'kind', 'year', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'certificates', 'rating', 'votes', 'cover url', 'plot outline', 'languages', 'akas', 'plot', 'canonical title', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', 'full-size cover url']
967429
When his brother is killed in a robbery, paraplegic Marine Jake Sully decides to take his place in a mission on the distant world of Pandora. There he learns of greedy corporate figurehead Parker Selfridge's intentions of driving off the native humanoid "Na'vi" in order to mine for the precious material scattered throughout their rich woodland. In exchange for the spinal surgery that will fix his legs, Jake gathers intel for the cooperating military unit spearheaded by gung-ho Colonel Quaritch, while simultaneously attempting to infiltrate the Na'vi people with the use of an "avatar" identity. While Jake

In [136]:
#177 = 8.0 Aladdin
#378 = 6.7 Robin Hood
m_temp[378]


'Robin Hood'

In [139]:
rating_avg.append(8.0)
rating_avg.append(6.7)


In [142]:
rating_avg
#vowel = ['a', 'e', 'i', 'u']
# inserting element to list at 4th position
#vowel.insert(3, 'o')
rating_avg.insert(177, 8.0)
rating_avg.insert(378, 6.7)


In [168]:
rating_avg_2 = rating_avg[:718]
df.head()

,name,rev,view,like,dislike,comment,genre,Distributor,Rating,Budget,Year
0,Avatar,2788.0,19929824,22396,2431,10455,Sci-Fi Adventure,Fox,PG-13,237.0,2009
1,Titanic,2187.5,16855757,39943,1823,10586,Romance,Paramount,PG-13,200.0,1997
2,Star Wars: The Force Awakens,2068.2,185150509,1097369,44323,209184,Sci-Fi Fantasy,Buena Vista,PG-13,245.0,2015
3,Jurassic World,1671.7,133246187,927462,47529,126958,Action / Adventure,Universal,PG-13,150.0,2015
4,Marvel's The Avengers,1518.8,143928901,2903270,62566,397740,Action / Adventure,Buena Vista,PG-13,220.0,2012


In [169]:
#set(df['name']) - set(m_temp)
df[df['name'] == "The Intouchables (U.S.-only)"]
rating_avg_2.insert(234,8.6)

In [178]:
df['rating'] = rating_avg_2
df.tail(5)
df.to_csv('all_data_movies_final.csv')